In [1]:
import tensorflow as tf
cifar = tf.keras.datasets.cifar10
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

CIFAR-10 is an established computer-vision dataset used for object recognition. It is a subset of the 80 million tiny images dataset and consists of 60,000 32x32 color images containing one of 10 object classes, with 6000 images per class.

In [2]:
# load cifar-10 dataset
(x_train, y_train),(x_test, y_test) = cifar.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

print("training shape: " + str(x_train.shape))
print("test shape: " + str(x_test.shape))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mac/.keras/datasets/cifar-10-batches-py/data_batch_1'

In [3]:
# imports the mobilenet model and discards the last layer.
# notice that we are also loading the pretrained weights trained with imagenet.
base_model=tf.keras.applications.vgg16.VGG16(weights='imagenet',include_top=False, input_shape=(32,32,3))

In [6]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [7]:
#Adding custom Layers 
x = base_model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
predictions = tf.keras.layers.Dense(10, activation="softmax")(x)

# creating the final model 
model = tf.keras.Model(inputs = base_model.input, outputs = predictions)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [8]:
# train only the top layers, set the base_model layers trainable attribute to False
# i.e. freeze all convolutional layers
# if you want to train the whole vgg16 network remove the for loop.
for layer in base_model.layers:
    layer.trainable = False
        
# different optimizers could be used, but doesn't have too much effect, perhaps fractions of 1 percent...
# also would make sense to check what optimizer was used when the VGG16 base model was trained.
model.compile(optimizer='adam',
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

In [9]:
# again if you get unexplainable errors, shutdown all other jupyter kernels (kernel->shutdown)
history = model.fit(x_train, y_train, validation_split=0.2, epochs=5)

NameError: name 'x_train' is not defined

In [8]:
# Run model model.evaluate with test data
model.evaluate(x_test, y_test)

NameError: name 'x_test' is not defined

In [9]:
# create a confusion matrix based on the x_test predictions compared to the true values y_test
# confusion matrix visualizes how many predictions where right and wrong and what was predicted.

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder

# create a prediction
y_pred = model.predict(x_test)

# create the confusion matrix
cm = confusion_matrix(y_test, y_pred.argmax(axis=1))

NameError: name 'x_test' is not defined

In [10]:
print(cm)

NameError: name 'cm' is not defined

In [11]:
# Visualize the confusion matrix

import seaborn as sn
import pandas  as pd
 
# rows are the predictions and columns the true labels. 
df_cm = pd.DataFrame(cm, ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck'],
                  ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck'])
# the size of each block
plt.figure(figsize = (10,7))
# for label size
sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True,annot_kws={"size": 12})# font size
plt.show()

NameError: name 'cm' is not defined

In [12]:
# list all data in history
print(history.history.keys())

NameError: name 'history' is not defined

In [13]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

## Do fine tuning for all layers

  * Run the notebook from beginning and remove the for loop "for layer in base_model.layers". Freeze only the first 4 layers?
  * use 10 times smaller learning rate so that the weights don't change too quickly: "adam = tf.keras.optimizers.Adam(lr=0.0001)". Imagenet preloaded weights are already pretty good.

## Exercices 

### Exercice 1: What if we had only 3000 training samples, how well would the network perform

Check next section how to generate more data when more data is not available.

### Exercise 2: Try to get as high test accuracy as possible

## Generating more training data

In [15]:
# prepare data augmentation configuration
# We will allow angle changes, zooming and horizontal flip
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [16]:
# This computes the internal data stats related to the
# data-dependent transformations, based on an array of sample data.
train_datagen.fit(x_train)

# Takes data & label arrays, generates batches of augmented data.
train_generator = train_datagen.flow(x_train, y_train, batch_size=32)

# do the same for validation data, no transformation is applied.
test_datagen = ImageDataGenerator()
validation_generator = test_datagen.flow(x_test, y_test, batch_size=32)

NameError: name 'x_train' is not defined

In [17]:
# train the model using the the defined generator.
model.fit_generator(
    train_generator,
    epochs=50,
    validation_data=validation_generator)

NameError: name 'model' is not defined

What else could we try. Do we really need all vgg16 layers? What if we would chop of more layers?

In [18]:
import matplotlib.image as mimg
import matplotlib.pyplot as plt
import numpy as np